In [48]:
from requests import get
import pandas as pd
import json


API_KEY = json.load(open("../../../Documents/api_keys.json", "r"))["yelp"]

response = get("https://api.yelp.com/v3/businesses/search",
                headers={"Authorization": "Bearer " + API_KEY},
                params={"location": "Urbana-Champaign", "limit": 50})

def parseCategories(categories):
    tags = ""
    for obj in categories:
        tags += obj["alias"] + ","
    return tags[:len(tags) - 1]

if response:
    df = pd.DataFrame.from_dict(response.json()["businesses"], orient="columns")
    
    # clean up some of the data formats
    df["address"] = df["location"].apply(lambda x: x["address1"])
    df["price"] = df["price"].apply(lambda x: len(x) * 10)
    
    # parse categories into an array of tags
    df["keywords"] = df["categories"].apply(parseCategories)
    
    # get only the columns we need
    df = df[["name", "phone", "address", "image_url", "price", "rating", "url", "keywords"]]
    
    json.dump(df.to_dict(orient="records"), open("../react-app/src/data/topPicks.json", "w"))

else:
    print("NOT FOUND")